In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os

# 1. Класс для загрузки датасета
class ChatDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.conversations = []

        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if '|||' in line:
                    question, answer = line.strip().split('|||', 1)
                    self.conversations.append((question.strip(), answer.strip()))

        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        question, answer = self.conversations[idx]
        text = f"Вопрос: {question}\nОтвет: {answer}"
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()
        }

# 2. Загрузка модели и токенизатора из интернета
def load_model_and_tokenizer(model_name="sberbank-ai/rugpt3small_based_on_gpt2"):
    print(f"Загрузка модели {model_name}...")
    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token

        model = GPT2LMHeadModel.from_pretrained(model_name)
        print("Модель и токенизатор успешно загружены!")
        return model, tokenizer
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        return None, None

# 3. Тонкая настройка модели
def fine_tune(model, tokenizer, dataset_path, output_dir="./fine_tuned_model"):
    # Проверка GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"Используется устройство: {device}")

    # Загрузка данных
    train_dataset = ChatDataset(dataset_path, tokenizer)
    print(f"Загружено примеров: {len(train_dataset)}")

    # Параметры обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=8,
        per_device_train_batch_size=4,
        save_steps=500,
        save_total_limit=2,
        learning_rate=5e-5,
        logging_steps=100,
        fp16=torch.cuda.is_available()
    )

    # Обучение
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    print("Начало обучения...")
    trainer.train()

    # Сохранение модели
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Модель сохранена в {output_dir}")

    return model, tokenizer

# 4. Генерация ответов
def generate_response(model, tokenizer, prompt, max_length=50):
    # Формируем промпт
    input_text = f"Вопрос: {prompt}\nОтвет:"

    # Токенизация с явным указанием attention_mask
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    ).to(model.device)

    # Генерация с "холодным" (более предсказуемым) алгоритмом
    output = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=len(inputs.input_ids[0]) + max_length,
        temperature=0.7,  # Уменьшаем "креативность"
        top_k=30,         # Ограничиваем словарь при генерации
        top_p=0.9,        # Nucleus sampling
        repetition_penalty=1.5,  # Штраф за повторения
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )

    # Извлекаем только ответ (после "Ответ:")
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = full_text.split("Ответ:")[-1].strip()

    # Ограничим ответ первым предложением
    if '.' in answer:
        answer = answer.split('.')[0] + '.'
    elif '!' in answer:
        answer = answer.split('!')[0] + '!'
    elif '?' in answer:
        answer = answer.split('?')[0] + '?'

    return answer

# 5. Основной цикл чата
def run_chatbot():
    # Загрузка модели (из интернета)
    model, tokenizer = load_model_and_tokenizer()

    if model is None:
        return

    # Проверка необходимости тонкой настройки
    if input("Выполнить тонкую настройку? (y/n): ").lower() == 'y':
        dataset_path = input("Введите путь к файлу с данными (dialogues.txt): ")
        model, tokenizer = fine_tune(model, tokenizer, dataset_path)

    # Чат-режим
    print("\nЧат-бот готов! Введите 'выход' для завершения.")
    while True:
        user_input = input("Вы: ")
        if user_input.lower() in ['выход', 'exit', 'quit']:
            break

        response = generate_response(model, tokenizer, user_input)
        print(f"Бот: {response}")

if __name__ == "__main__":
    run_chatbot()

Загрузка модели sberbank-ai/rugpt3small_based_on_gpt2...
Модель и токенизатор успешно загружены!
Выполнить тонкую настройку? (y/n): y
Введите путь к файлу с данными (dialogues.txt): /content/q and a.txt
Используется устройство: cpu
Загружено примеров: 2
Начало обучения...


Step,Training Loss


Модель сохранена в ./fine_tuned_model

Чат-бот готов! Введите 'выход' для завершения.
Вы: привет
Бот: если в тексте вопроса есть опечатка, то это ошибка.
Вы: привет
Бот: Привет! Не знаю как Вам, но я думаю что это не так.
Вы: как дела?
Бот: Как и все.
Вы: выход
